# Course-End Project: Task Manager with User Authentication

## Problem Statement:
### In today’s world, individuals often need to keep track of various tasks in a structured
### way. You are tasked with building a Task Manager that allows users to manage their
### tasks. The system should include user authentication, meaning each user has to log
### in with a username and password. Once logged in, users can create, view, update,
### and delete their tasks. Each user’s tasks should be stored separately, and only the
### authenticated user can access their tasks.

## Objectives:
### 1. Design and implement a user authentication system (login and registration)
### 2. Create a task management system that allows users to:
### a. Add, view, mark as completed, and delete tasks
### 3. Use file handling to store user credentials and tasks persistently
### 4. Create an interactive menu-driven interface to manage tasks

In [63]:
### Design and implement a user authentication system (login and registration)

import os
import json
import hashlib
import csv
from datetime import datetime

class User:

    def __init__(self, name, username, password):
        self.__name = name
        self.__username = username
        self.__password = self.__hashPassword(password)

    @staticmethod
    def __hashPassword(password):
        return hashlib.sha256(password.encode()).hexdigest()

    @property
    def name(self):
        return self.__name

    @name.setter
    def name(self, value):
        self.__name = value

    @property
    def username(self):
        return self.__username

    @property
    def password(self):
        return self.__password

    @password.setter
    def password(self, value):
        self.__password = self.__hashPassword(value)

    @staticmethod
    def loadUsers():
        if not os.path.exists('users.json'):
            return {}
        with open('users.json', 'r') as file:
            return json.load(file)

    @staticmethod
    def saveUsers(users):
        with open('users.json', 'w') as file:
            json.dump(users, file)

    def registerUser(self):
        users = self.loadUsers()
        if self.__username in users:
            return False, "Username already exists"
        users[self.__username] = self.__password
        User.saveUsers(users)
        return True, "Registration successful"

    def loginUser(self):
        users = self.loadUsers()
        if self.__username in users and users[self.__username] == self.__password:
            return True, "Login successful"
        return False, "Invalid username or password"

    def deleteAccount(self):
        users = self.loadUsers()
        if self.__username in users:
            del users[self.__username]
            User.saveUsers(users)
            if os.path.exists(f'{self.__username}_tasks.json'):
                os.remove(f'{self.__username}_tasks.json')
            return True, "Account deleted successfully"
        return False, "Account not found"

In [65]:
class TaskManager:
    
    def __init__(self, username):
        self.__username = username
        self.__tasks = self.loadTasks()

    @property
    def username(self):
        return self.__username

    @username.setter
    def username(self, value):
        self.__username = value

    @property
    def tasks(self):
        return self.__tasks

    @tasks.setter
    def tasks(self, value):
        self.__tasks = value

    def addTask(self, taskDescription):
        task_id = len(self.__tasks) + 1
        task = {'id': task_id, 'description': taskDescription, 'completed': False}
        self.__tasks.append(task)
        self.saveTasks()
        return "Task added successfully."

    def viewTasks(self):
        return [{'id': task['id'], 'description': task['description'], 
                 'status': 'Completed' if task['completed'] else 'Pending'} for task in self.__tasks]
        
    def completeTask(self, taskId):
        for task in self.__tasks:
            if task['id'] == taskId:
                task['completed'] = True
                self.saveTasks()
                return "Task marked as completed."
        return "Task not found."

    def deleteTask(self, taskId):
        for index, task in enumerate(self.__tasks):
            if task['id'] == taskId:
                del self.__tasks[index]
                self.saveTasks()
                return "Task deleted successfully."
        return "Task not found."

    def loadTasks(self):
        if not os.path.exists(f'{self.__username}_tasks.json'):
            return []
        with open(f'{self.__username}_tasks.json', 'r') as file:
            return json.load(file)

    def saveTasks(self):
        with open(f'{self.__username}_tasks.json', 'w') as file:
            json.dump(self.__tasks, file)

In [67]:
class BudgetManager:

    def __init__(self, username):
        self.__username = username
        self.__monthlyBudget = 0
        self.__expenses = self.loadExpenses()

    @property
    def monthlyBudget(self):
        return self.__monthlyBudget

    @monthlyBudget.setter
    def monthlyBudget(self, value):
        self.__monthlyBudget = value

    def addExpense(self, category, amount, description):
        expense = {
            'date': datetime.now().strftime('%Y-%m-%d'),
            'category': category,
            'amount': amount,
            'description': description
        }
        self.__expenses.append(expense)
        self.saveExpenses()

    def viewExpenses(self):
        return self.__expenses

    def calculateTotalExpenses(self):
        return sum(expense['amount'] for expense in self.__expenses)

    def checkBudget(self):
        totalExpenses = self.calculateTotalExpenses()
        if totalExpenses > self.__monthlyBudget:
            return False, f"You have exceeded your budget by ${totalExpenses - self.__monthlyBudget}!"
        else:
            return True, f"You have ${self.__monthlyBudget - totalExpenses} left for the month."

    def loadExpenses(self):
        if not os.path.exists(f'{self.__username}_expenses.csv'):
            return []
        with open(f'{self.__username}_expenses.csv', 'r') as file:
            reader = csv.DictReader(file)
            expenses = []
            for row in reader:
                row['amount'] = float(row['amount'])
                expenses.append(row)
            return expenses

    def saveExpenses(self):
        with open(f'{self.__username}_expenses.csv', 'w', newline='') as file:
            fieldnames = ['date', 'category', 'amount', 'description']
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(self.__expenses)

In [85]:
class Application:

    def __init__(self):
        self.__currentUser = None
        self.__taskManager = None
        self.__budgetManager = None

    def mainMenu(self):
        print("Welcome to Task and Budget Manager!")
        while True:
            choice = input("1. Register\n2. Login\n3. Exit\nChoose an option: ")
            if choice == '1':
                self.registerUser()
            elif choice == '2':
                self.loginUser()
            elif choice == '3':
                break

    def registerUser(self):
        name = input("Enter your name: ")
        username = input("Enter username: ")
        password = input("Enter password: ")
        user = User(name, username, password)
        success, message = user.registerUser()
        print(message)

    def loginUser(self):
        username = input("Enter username: ")
        password = input("Enter password: ")
        user = User("", username, password)  # Name is not needed for login
        success, message = user.loginUser()
        if success:
            print(message)
            self.__currentUser = username
            self.__taskManager = TaskManager(username)
            self.__budgetManager = BudgetManager(username)
            self.taskMenu()
        else:
            print(message)

    def taskMenu(self):
        while True:
            choice = int(input("1. Add Task\n2. View Tasks\n3. Complete Task\n4. Delete Task\n5. Set Monthly Budget\n" + 
                               "6. Add Expense\n7. View Expenses\n8. Check Budget\n9. Logout\nChoose an option: "))
            if choice == 1:
                taskDescription = input("Enter task description: ")
                print(self.__taskManager.addTask(taskDescription))
            elif choice == 2:
                tasks = self.__taskManager.viewTasks()
                for task in tasks:
                    print(f"ID: {task['id']}, {task['description']} - {task['status']}")
            elif choice == 3:
                taskId = int(input("Enter task ID to complete: "))
                print(self.__taskManager.completeTask(taskId))
            elif choice == 4:
                taskId = int(input("Enter task ID to delete: "))
                print(self.__taskManager.deleteTask(taskId))
            elif choice == 5:
                budget = float(input("Enter your monthly budget: "))
                self.__budgetManager.monthlyBudget = budget
                print("Monthly budget set.")
            elif choice == 6:
                category = input("Enter expense category: ")
                amount = float(input("Enter expense amount: "))
                description = input("Enter expense description: ")
                self.__budgetManager.addExpense(category, amount, description)
                print("Expense added successfully.")
            elif choice == 7:
                expenses = self.__budgetManager.viewExpenses()
                for expense in expenses:
                    print(f"Date: {expense['date']}, Category: {expense['category']}, Amount: ${expense['amount']}, "
                          f"Description: {expense['description']}")
            elif choice == 8:
                isWithinBudget, message = self.__budgetManager.checkBudget()
                print(message)
            elif choice == 9:
                break

In [71]:
# if __name__ == "__main__":
#     app = Application()
#     app.mainMenu()

In [73]:
# Simulate user registration
name = "John Doe"
username = "johndoe"
password = "password123"
user = User(name, username, password)
success, message = user.registerUser()
print(message)

Registration successful


In [75]:
# Simulate user login
user = User("", username, password)
success, message = user.loginUser()
print(message)

Login successful


In [77]:
# Initialize task manager for the logged-in user
task_manager = TaskManager(username)

# Add a task
task_description = "Complete project report"
print(task_manager.addTask(task_description))

# View tasks
tasks = task_manager.viewTasks()  # This now returns the status
for task in tasks:
    print(f"ID: {task['id']}, Description: {task['description']} - Status: {task['status']}")  # Change 'completed' to 'status'

# Complete a task
task_id = 1  # Assuming the task ID is 1
print(task_manager.completeTask(task_id))

# View tasks again
tasks = task_manager.viewTasks()
for task in tasks:
    print(f"ID: {task['id']}, Description: {task['description']} - Status: {task['status']}")  # Change 'completed' to 'status'

# Delete a task
print(task_manager.deleteTask(task_id))

Task added successfully.
ID: 1, Description: Complete project report - Status: Pending
Task marked as completed.
ID: 1, Description: Complete project report - Status: Completed
Task deleted successfully.


In [79]:
# Initialize the budget manager for the logged-in user
budget_manager = BudgetManager(username)

# Set monthly budget
budget_manager.monthlyBudget = 500
print("Monthly budget set.")

# Add an expense
category = "Food"
amount = 50  # Make sure this is a numeric value
description = "Lunch with friends"
budget_manager.addExpense(category, amount, description)
print("Expense added successfully.")

# View expenses
expenses = budget_manager.viewExpenses()
for expense in expenses:
    print(f"Date: {expense['date']}, Category: {expense['category']}, Amount: ${expense['amount']}, Description: {expense['description']}")

# Check budget
is_within_budget, message = budget_manager.checkBudget()
print(message)

Monthly budget set.
Expense added successfully.
Date: 2024-10-19, Category: Food, Amount: $50, Description: Lunch with friends
You have $450 left for the month.
